In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
sc =SparkContext()

In [13]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf, lit
from pyspark.sql.types import IntegerType
from pyspark import SQLContext
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml import Pipeline
import pandas as pd

spark = SparkSession\
    .builder\
    .appName("Clustering")\
    .getOrCreate()
# dataframe1 = spark.read.json('hdfs://localhost:1234/user/tl2861/hw3/train.json')

sqlContext = SQLContext(sc)
# dataframe1 = sqlContext.read.json('hdfs://localhost:1234/user/tl2861/hw3/train.json')
dataframe1 = sqlContext.read.json('../../../data/AA/wiki_*').withColumn('label',lit(0))
dataframe2 = sqlContext.read.json('../../../data/AAquote/wiki_*').withColumn('label', lit(1))
# dataframe3 = sqlContext.read.json('../../data/AAvoyage/wiki_*').withColumn('label',lit(3))
dataframe = dataframe1.unionAll(dataframe2)
# dataframe1.printSchema()
# dataframe2.printSchema()
dataframe.printSchema()
# dataframe1.show()
#regexTokenizer = RegexTokenizer()
sampledData = dataframe.sampleBy("label", fractions={0: 0.2, 1: 0.2}, seed=0)
# sampledData.show()
# sampledData0 = pd.DataFrame(sampledData)

regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="[^A-Za-z]+", toLowercase=True)
tokenizedData = regexTokenizer.transform(sampledData)

stopWordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
filteredData = stopWordsRemover.transform(tokenizedData)

hashingTF = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=20)
featurizedData = hashingTF.transform(filteredData)

idf= IDF(inputCol="raw_features", outputCol="features")
idfModel = idf.fit(featurizedData)
data1 = idfModel.transform(featurizedData)

datatext = data1.select('id','features','label')
datatext.show()

root
 |-- id: string (nullable = true)
 |-- text: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- label: integer (nullable = false)

+------+--------------------+-----+
|    id|            features|label|
+------+--------------------+-----+
|205661|(20,[0,1,2,3,4,5,...|    0|
|206148|(20,[0,1,2,3,4,5,...|    0|
|206369|(20,[13],[0.33957...|    0|
|206436|(20,[0,1,2,3,4,5,...|    0|
|207119|(20,[0,1,2,3,4,5,...|    0|
|207153|(20,[0,1,2,3,4,5,...|    0|
|207214|(20,[0,1,2,3,4,5,...|    0|
|207437|(20,[0,1,2,3,4,5,...|    0|
|207482|(20,[0,1,2,3,4,5,...|    0|
|207735|(20,[0,1,2,3,4,5,...|    0|
|208088|(20,[0,1,2,3,4,5,...|    0|
|208519|(20,[0,1,2,3,4,5,...|    0|
|208803|(20,[0,1,2,3,4,5,...|    0|
|208933|(20,[0,1,2,3,4,5,...|    0|
|209059|(20,[0,1,2,3,4,5,...|    0|
|209467|(20,[0,1,2,3,4,5,...|    0|
|209844|(20,[0,1,2,3,4,5,...|    0|
|210549|(20,[0,2,3,4,5,6,...|    0|
|210940|(20,[0,1,2,3,4,5,...|    0|
|210997|(20,[0,1,2,3,

In [ ]:
datatext.count()

In [14]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[regexTokenizer, stopWordsRemover, hashingTF,idf])
pipeline_model=pipeline.fit(sampledData)
processed_data = pipeline_model.transform(sampledData)

In [27]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import ArrayType, DoubleType

def to_array(col):
    def to_array_(v):
        return v.toArray().tolist()
    return udf(to_array_, ArrayType(DoubleType()))(col)

datatext_plain = datatext.withColumn("feature", to_array(col("features")))\
    .select(["id"] + [col("feature")[i] for i in range(20)] +["label"])

In [24]:
datatext_plain.toPandas().to_csv("../1/dataCluster.csv", sep = ",",index = False, encoding = "utf-8")

this is the end of a file

In [9]:
import numpy as np
datatext_np = np.array(datatext)
# transform pandas to np array
print(datatext_np)

[['205661'
  SparseVector(20, {0: 2.7814, 1: 1.1255, 2: 1.1968, 3: 3.0374, 4: 3.7715, 5: 1.453, 6: 0.2845, 7: 0.3667, 8: 3.2155, 9: 2.9031, 10: 1.8492, 11: 2.9315, 12: 1.4754, 13: 1.6979, 14: 1.8128, 15: 0.6476, 16: 0.6827, 17: 2.8028, 18: 3.051, 19: 2.2871})
  0]
 ['206148'
  SparseVector(20, {0: 6.2582, 1: 3.7516, 2: 5.5849, 3: 3.0374, 4: 7.1658, 5: 3.6326, 6: 9.1048, 7: 8.0674, 8: 3.93, 9: 4.1933, 10: 4.3147, 11: 2.5651, 12: 4.1803, 13: 11.206, 14: 6.0426, 15: 5.8288, 16: 2.731, 17: 4.9828, 18: 6.1019, 19: 2.6138})
  0]
 ['206369' SparseVector(20, {13: 0.3396}) 0]
 ...
 ['209579'
  SparseVector(20, {0: 0.6954, 1: 0.7503, 2: 0.3989, 3: 0.9112, 4: 0.7543, 5: 0.9687, 6: 0.2845, 8: 0.3573, 9: 0.9677, 10: 0.9246, 12: 0.9836, 13: 0.6792, 17: 0.6228, 18: 0.3051})
  1]
 ['209581'
  SparseVector(20, {0: 0.3477, 3: 1.2149, 5: 0.4843, 7: 0.7334, 8: 0.3573, 9: 0.3226, 12: 0.7377, 14: 0.6043, 16: 0.6827})
  1]
 ['209594'
  SparseVector(20, {2: 0.3989, 4: 0.7543, 8: 0.3573, 12: 0.9836, 14: 0.3021

In [11]:
featureCol = datatext[:,1]
print (featureCol)

[SparseVector(20, {0: 2.7814, 1: 1.1255, 2: 1.1968, 3: 3.0374, 4: 3.7715, 5: 1.453, 6: 0.2845, 7: 0.3667, 8: 3.2155, 9: 2.9031, 10: 1.8492, 11: 2.9315, 12: 1.4754, 13: 1.6979, 14: 1.8128, 15: 0.6476, 16: 0.6827, 17: 2.8028, 18: 3.051, 19: 2.2871})
 SparseVector(20, {0: 6.2582, 1: 3.7516, 2: 5.5849, 3: 3.0374, 4: 7.1658, 5: 3.6326, 6: 9.1048, 7: 8.0674, 8: 3.93, 9: 4.1933, 10: 4.3147, 11: 2.5651, 12: 4.1803, 13: 11.206, 14: 6.0426, 15: 5.8288, 16: 2.731, 17: 4.9828, 18: 6.1019, 19: 2.6138})
 SparseVector(20, {13: 0.3396}) ...
 SparseVector(20, {0: 0.6954, 1: 0.7503, 2: 0.3989, 3: 0.9112, 4: 0.7543, 5: 0.9687, 6: 0.2845, 8: 0.3573, 9: 0.9677, 10: 0.9246, 12: 0.9836, 13: 0.6792, 17: 0.6228, 18: 0.3051})
 SparseVector(20, {0: 0.3477, 3: 1.2149, 5: 0.4843, 7: 0.7334, 8: 0.3573, 9: 0.3226, 12: 0.7377, 14: 0.6043, 16: 0.6827})
 SparseVector(20, {2: 0.3989, 4: 0.7543, 8: 0.3573, 12: 0.9836, 14: 0.3021, 15: 0.6476})]


In [5]:
datatext_np.toPandas().to_csv("../1/dataCluster.csv", sep = ",",index = False, encoding = "utf-8")

#### Processing the clustering data

In [22]:
from pyspark.ml.clustering import KMeans
kmeans = KMeans().setK(2).setSeed(4381)
kmModel = kmeans.fit(data1)
wssse = kmModel.computeCost(data1)
print("Within Set Sum of Squared Errors = " + str(wssse))
centers = kmModel.clusterCenters()
print("Cluster Centers: ")
for center in centers:
     print(center)
        
data_kmeans = kmModel.transform(datatext).select("id","features","prediction")
     
datakmeans_plain = data_kmeans.withColumn("feature", to_array(col("features")))\
    .select(["id"] + [col("feature") [i] for i in range(20)] +["prediction"])
datakmeans_plain.show() 
datakmeans_plain.toPandas().to_csv("../1/dataCluster-kmeans.csv",sep = ",", index = False, encoding = "utf-8") 

Within Set Sum of Squared Errors = 1335214.9809889607
Cluster Centers: 
[24.70595957 21.86310698 28.09276079 24.66477462 31.0944174  18.56860122
 22.48443747 19.4617212  23.95028956 18.58052216 23.5488747  21.43006121
 17.48836579 26.64247329 20.18451575 24.49362609 19.96815644 22.04731022
 19.89367364 21.86714012]
[1.45820232 1.27541215 1.43199953 1.33120029 1.53704863 1.16367569
 1.33369765 1.24050032 1.49248339 1.28095416 1.32525024 1.31310688
 1.16756241 1.47920655 1.32047555 1.39996726 1.30438661 1.2579419
 1.25805494 1.19740135]
+------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+----------+
|    id|        feature[0]|        feature[1]|        

In [40]:
# generate clusters
import numpy as np
# mean 0 
label1 = np.full((100,1),1,dtype=int)
# label1.astype(int)
label2 = np.full((100,1),2,dtype=int)
label3 = np.full((100,1),3,dtype=int)
array1 = np.random.multivariate_normal([0]*20,np.identity(20),100)
array1 = np.concatenate([array1,label1],axis = 1)
# mean 100
array2 = np.random.multivariate_normal([100]*20,np.identity(20),100)
array2 = np.concatenate([array2,label2],axis = 1)
# mean 200
array3 = np.random.multivariate_normal([200]*20,np.identity(20),100)
array3 = np.concatenate([array3,label3],axis = 1)
array = np.concatenate([array1,array2,array3],axis = 0)
print (array)

[[-7.89559931e-01  7.70555337e-01 -4.35653554e-01 ...  1.44903782e+00
  -1.14509974e+00  1.00000000e+00]
 [-7.13219895e-01  1.20640230e+00 -2.17015851e-01 ... -5.33261729e-01
   9.80583990e-01  1.00000000e+00]
 [-3.63368308e-01 -3.35528009e-01 -1.04976740e+00 ... -1.20101254e-01
   6.45435412e-01  1.00000000e+00]
 ...
 [ 1.99597173e+02  1.99965867e+02  2.00999524e+02 ...  2.01284446e+02
   1.99332399e+02  3.00000000e+00]
 [ 2.01424832e+02  1.99586398e+02  2.00772361e+02 ...  2.00295147e+02
   1.98874812e+02  3.00000000e+00]
 [ 2.00761509e+02  2.01134890e+02  2.00306142e+02 ...  1.98432821e+02
   1.99848450e+02  3.00000000e+00]]


In [41]:
columnstr = []
for i in range(array.shape[1]-1):
    columnstr.append("feature{}".format(i))
columnstr.append("label")
print(columnstr)
randomdf = pd.DataFrame(array,columns=columnstr)
randomdf.insert(0,"id",randomdf.index)
randomdf[["label"]].astype("int64")
display(randomdf)
randomdf.to_csv("../1/random.csv",sep = ",", index = False, encoding = "utf-8")

['feature0', 'feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6', 'feature7', 'feature8', 'feature9', 'feature10', 'feature11', 'feature12', 'feature13', 'feature14', 'feature15', 'feature16', 'feature17', 'feature18', 'feature19', 'label']


,id,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,...,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,label
0,0,-0.789560,0.770555,-0.435654,-0.930807,2.536585,-1.105072,2.000391,0.957748,-0.452267,...,0.538354,1.561798,-0.348898,-0.728021,0.917681,0.125946,1.688935,1.449038,-1.145100,1.0
1,1,-0.713220,1.206402,-0.217016,-1.641159,-1.757719,-0.636980,0.239440,-1.137837,-0.825197,...,-1.744764,-0.082631,1.312835,-1.011670,0.431885,-0.552218,1.514978,-0.533262,0.980584,1.0
2,2,-0.363368,-0.335528,-1.049767,-0.005734,0.105387,-0.116409,0.654993,1.808842,-0.423882,...,1.037314,1.360705,0.686361,-0.460339,0.087342,-0.441700,1.376119,-0.120101,0.645435,1.0
3,3,1.840168,0.776492,-0.086645,0.403888,0.467253,0.853783,0.691137,0.966216,0.511907,...,-1.900358,0.585617,0.443978,-0.650174,-0.754919,2.154892,-0.765595,-0.285966,0.018802,1.0
4,4,-1.041724,-2.104703,-0.261017,0.759860,-0.360700,-1.113902,0.330424,2.384186,0.735269,...,0.852157,-1.595742,0.545070,0.960059,-1.286117,-0.265234,0.531972,-0.280312,0.420358,1.0
5,5,-0.812214,0.062742,1.394079,-0.741141,0.999505,-1.461588,0.474784,-0.196416,-1.097829,...,-0.949991,-0.591894,-0.196282,-0.506970,-0.780401,0.972983,0.371214,-0.679687,2.068463,1.0
6,6,0.620793,0.303373,-0.120447,1.646381,0.197358,-0.472187,-1.084097,-0.276402,-0.146689,...,0.393749,0.045491,0.095373,-0.852639,0.134026,-3.547737,0.681443,-1.078338,0.267743,1.0
7,7,0.839716,1.232522,-0.127380,0.993125,0.997933,-0.290397,-0.564638,0.691065,2.682128,...,0.705024,-0.083150,1.745659,1.688070,-1.183051,0.029006,0.668348,-1.300560,1.169204,1.0
8,8,-0.580265,1.096712,-0.349437,0.396693,0.820282,-1.704255,0.335280,-1.208231,-1.223845,...,-0.294582,-0.313546,0.763162,-0.687848,1.029566,-0.832131,0.526189,0.616997,0.959476,1.0
9,9,0.997475,-1.554936,1.430824,0.258315,-0.247030,0.142118,0.091324,0.887235,1.402242,...,1.038226,1.302626,0.718447,0.016617,1.050622,-1.162702,-1.732393,-0.239256,-0.341978,1.0


## Another data processing method - get feature

In [10]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf, lit
from pyspark.sql.types import IntegerType
from pyspark import SQLContext
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml import Pipeline
import pandas as pd

spark = SparkSession\
    .builder\
    .appName("Clustering")\
    .getOrCreate()
# dataframe1 = spark.read.json('hdfs://localhost:1234/user/tl2861/hw3/train.json')

sqlContext = SQLContext(sc)
# dataframe1 = sqlContext.read.json('hdfs://localhost:1234/user/tl2861/hw3/train.json')
dataframe1 = sqlContext.read.json('../../../data/AA/wiki_*').withColumn('label',lit(0))
dataframe2 = sqlContext.read.json('../../../data/AAquote/wiki_*').withColumn('label', lit(1))
dataframe3 = sqlContext.read.json('../../../data/AAvoyage/wiki_*').withColumn('label',lit(2))
dataframe = dataframe1.unionAll(dataframe2).unionAll(dataframe3)
# dataframe1.printSchema()
# dataframe2.printSchema()
dataframe.printSchema()
# dataframe1.show()
#regexTokenizer = RegexTokenizer()
sampledData = dataframe.sampleBy("label", fractions={0: 0.3, 1: 0.2, 2:0.22}, seed=0)
sampledData.groupBy("label").count().orderBy("label").show()
# sampledData.show()
# sampledData0 = pd.DataFrame(sampledData)

root
 |-- id: string (nullable = true)
 |-- text: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- label: integer (nullable = false)

+-----+-----+
|label|count|
+-----+-----+
|    0| 6536|
|    1| 6586|
|    2| 6252|
+-----+-----+



In [104]:
from pyspark.ml.feature import Word2Vec
def preprocessing(data):
    regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="[^A-Za-z]+", toLowercase=True)
#     tokenizedData = regexTokenizer.transform(sampledData)

    stopWordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
#     filteredData = stopWordsRemover.transform(tokenizedData)

    word2Vec = Word2Vec(vectorSize = 20, minCount = 0, inputCol = "filtered_words", outputCol = "features")
#     hashingTF = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=20)
#     featurizedData = hashingTF.transform(filteredData)

#     idf= IDF(inputCol="raw_features", ou tputCol="features")
#     idfModel = idf.fit(featurizedData)
#     data1 = idfModel.transform(featurizedData)
    pipeline = Pipeline(stages=[regexTokenizer, stopWordsRemover, word2Vec])
    pipeline_model = pipeline.fit(data)
    processed_data = pipeline_model.transform(data)
    processed_data.show(5)
    return processed_data

processed_data = preprocessing(sampledData)
data_final = processed_data.select('id','features','label')
data_final.show()

+------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+
|    id|                text|               title|                 url|label|               words|      filtered_words|            features|
+------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+
|205661|Australian rules ...|Australian rules ...|https://en.wikine...|    0|[australian, rule...|[australian, rule...|[0.01114924066442...|
|206042|Tea Party-endorse...|Tea Party-endorse...|https://en.wikine...|    0|[tea, party, endo...|[tea, party, endo...|[-0.2171006524381...|
|206148|Pope Benedict XVI...|Pope Benedict XVI...|https://en.wikine...|    0|[pope, benedict, ...|[pope, benedict, ...|[-0.1306628265496...|
|206369|           Victoria
|            Victoria|https://en.wikine...|    0|          [victoria]|          [victoria]|[0.04427373409271...|
|206436|Gunma

### clustering

In [105]:
from pyspark.ml.clustering import KMeans
kmeans = KMeans().setK(3).setSeed(0)
kmModel = kmeans.fit(data_final)
wssse = kmModel.computeCost(data_final)
print("Within Set Sum of Squared Errors = " + str(wssse))
centers = kmModel.clusterCenters()
print("Cluster Centers: ")
for center in centers:
     print(center)

Within Set Sum of Squared Errors = 4771.550970753334
Cluster Centers: 
[-0.06746295  0.07249205  0.01577213 -0.00033322  0.03566203 -0.08792451
 -0.03205854 -0.11394079 -0.04191684 -0.0393172  -0.07674289 -0.05938614
 -0.00354545  0.09770459 -0.23672154 -0.06960879  0.10570871 -0.06008687
 -0.08971855 -0.01067245]
[ 0.06864628 -0.11439167  0.18817531  0.02348565  0.10188966  0.00826426
  0.0391198  -0.04795329 -0.00339289 -0.18372466 -0.30967515  0.10654605
 -0.16150138  0.20861526 -0.249925    0.05092078  0.18912806 -0.05546258
 -0.18679167 -0.08909115]
[ 0.24855162 -0.11964465  0.22945423 -0.20086624  0.05722153 -0.10261901
 -0.15050837 -0.03675126 -0.04186367 -0.19958509 -0.00775226 -0.06453671
 -0.3662974  -0.29472784 -0.32231585 -0.58864435  0.21580529 -0.29655021
 -0.57117251  0.36686592]


In [111]:
data_cluster = kmModel.transform(data_final).select("label","prediction")
sorted(data_cluster.groupBy("label","prediction").count().collect())

[Row(label=0, prediction=0, count=5698),
 Row(label=0, prediction=1, count=827),
 Row(label=0, prediction=2, count=11),
 Row(label=1, prediction=0, count=5977),
 Row(label=1, prediction=1, count=297),
 Row(label=1, prediction=2, count=312),
 Row(label=2, prediction=0, count=168),
 Row(label=2, prediction=1, count=6084)]

#### Split features

In [72]:
import numpy as np
data_np_feature = np.array(data_final.select("features").collect())
print (data_np_feature)

[[[ 0.01114924  0.11676429  0.00869855 ... -0.2043883  -0.31009984
   -0.41163858]]

 [[-0.21710065  0.06293082 -0.06669668 ... -0.15459449 -0.02849022
   -0.12072804]]

 [[-0.13066283  0.03310324  0.0528255  ... -0.09313601 -0.09908256
   -0.0828023 ]]

 ...

 [[ 0.16807088 -0.18440218  0.20753857 ... -0.07309774 -0.25222754
   -0.02921101]]

 [[ 0.03205101 -0.29853515  0.10677899 ... -0.07317124 -0.15665666
   -0.19225134]]

 [[ 0.03632184  0.1679611   0.10989097 ...  0.30429685 -0.08245945
    0.12173656]]]


In [74]:
data_np_feature.shape

(19374, 1, 20)

In [76]:
data_feature = np.squeeze(data_np_feature)

In [114]:
# data_id = np.array(data_final.select("id").collect())
data_label = np.array(data_final.select("label").collect())
data_prediction = np.array(data_cluster.select("prediction").collect())

In [119]:
data_withlabel = np.concatenate((data_feature,data_label),axis = 1)
print(data_withlabel.shape)
data_withpre = np.concatenate((data_feature,data_prediction), axis = 1)
print(data_withpre.shape)

(19374, 21)
(19374, 21)


In [115]:
data_df = pd.DataFrame(data_withlabel)
data_df.insert(0,"id",data_df.index)
# display(data_df)
columns = {x:'feature' + str(x) for x in range(20)}
columns[20] = 'label'
print (columns)
data_df_rename = data_df.rename(columns, axis = 'columns')
display(data_df_rename)
data_df_rename.to_csv("../1/cluster_label_2.csv",sep = ",", index = False, encoding = "utf-8")

{0: 'feature0', 1: 'feature1', 2: 'feature2', 3: 'feature3', 4: 'feature4', 5: 'feature5', 6: 'feature6', 7: 'feature7', 8: 'feature8', 9: 'feature9', 10: 'feature10', 11: 'feature11', 12: 'feature12', 13: 'feature13', 14: 'feature14', 15: 'feature15', 16: 'feature16', 17: 'feature17', 18: 'feature18', 19: 'feature19', 20: 'label'}


,id,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,...,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,label
0,0,0.011149,0.116764,0.008699,0.162206,0.053647,-0.350122,-0.175407,-0.060272,-0.096332,...,0.095576,0.066470,0.106227,0.092146,-0.213147,0.184564,-0.204388,-0.310100,-0.411639,0.0
1,1,-0.217101,0.062931,-0.066697,-0.160883,0.077830,-0.049406,-0.107326,-0.255011,0.116582,...,0.142249,0.130675,-0.165637,-0.278567,-0.059666,0.211337,-0.154594,-0.028490,-0.120728,0.0
2,2,-0.130663,0.033103,0.052826,0.016735,0.146488,-0.113662,0.060567,-0.116804,-0.034527,...,-0.073000,-0.044234,0.053423,-0.313691,-0.018427,0.125421,-0.093136,-0.099083,-0.082802,0.0
3,3,0.044274,-0.585843,0.104271,0.043293,0.361709,-0.330513,0.173843,0.097807,-0.438939,...,0.307055,-0.335232,0.200385,-0.227034,-0.169574,0.169691,-0.381725,-0.275363,-0.253356,0.0
4,4,-0.061785,0.041214,-0.007492,-0.095529,0.144389,-0.073329,0.065868,0.044003,-0.194689,...,-0.236539,-0.111709,0.097663,-0.161093,0.065875,0.131907,0.097411,-0.127102,0.054436,0.0
5,5,-0.172550,0.065910,0.097618,0.027284,0.099513,-0.087045,0.022871,0.008577,-0.073297,...,-0.074112,0.069528,-0.004766,-0.315807,-0.023256,0.189096,-0.124768,-0.143838,-0.040053,0.0
6,6,0.065295,0.188122,0.264526,0.052438,-0.016906,-0.017192,0.033818,-0.077562,0.076214,...,-0.019048,-0.215798,0.030529,-0.148937,0.028558,0.103843,0.047945,-0.282794,0.000171,0.0
7,7,-0.019264,0.146217,0.101780,-0.020692,-0.095936,0.039669,0.041793,-0.085271,-0.041021,...,-0.079179,-0.110792,0.133936,-0.149333,0.128809,0.090958,0.012027,-0.149558,-0.044397,0.0
8,8,0.078102,0.024764,0.222488,-0.007545,0.110410,-0.070361,0.165535,0.018249,0.074530,...,0.027959,-0.215296,0.111713,-0.248692,0.051753,0.216604,-0.059347,-0.282305,-0.098882,0.0
9,9,-0.053129,0.143064,0.090162,0.018056,0.074839,-0.001311,0.040774,-0.048419,-0.068145,...,-0.148575,-0.028270,0.095499,-0.127128,-0.006080,0.205171,0.047120,-0.112574,0.060608,0.0


In [121]:
data_pre_df = pd.DataFrame(data_withpre)
data_pre_df.insert(0,"id",data_pre_df.index)
columns_pre = {x:'feature' + str(x) for x in range(20)}
columns_pre[20] = 'prediction'
print (columns_pre)
data_pre_df_rename = data_df.rename(columns_pre, axis = 'columns')
display(data_pre_df_rename)
data_pre_df_rename.to_csv("../1/cluster_prediction_2.csv",sep = ",", index = False, encoding = "utf-8")

{0: 'feature0', 1: 'feature1', 2: 'feature2', 3: 'feature3', 4: 'feature4', 5: 'feature5', 6: 'feature6', 7: 'feature7', 8: 'feature8', 9: 'feature9', 10: 'feature10', 11: 'feature11', 12: 'feature12', 13: 'feature13', 14: 'feature14', 15: 'feature15', 16: 'feature16', 17: 'feature17', 18: 'feature18', 19: 'feature19', 20: 'prediction'}


,id,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,...,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,prediction
0,0,0.011149,0.116764,0.008699,0.162206,0.053647,-0.350122,-0.175407,-0.060272,-0.096332,...,0.095576,0.066470,0.106227,0.092146,-0.213147,0.184564,-0.204388,-0.310100,-0.411639,0.0
1,1,-0.217101,0.062931,-0.066697,-0.160883,0.077830,-0.049406,-0.107326,-0.255011,0.116582,...,0.142249,0.130675,-0.165637,-0.278567,-0.059666,0.211337,-0.154594,-0.028490,-0.120728,0.0
2,2,-0.130663,0.033103,0.052826,0.016735,0.146488,-0.113662,0.060567,-0.116804,-0.034527,...,-0.073000,-0.044234,0.053423,-0.313691,-0.018427,0.125421,-0.093136,-0.099083,-0.082802,0.0
3,3,0.044274,-0.585843,0.104271,0.043293,0.361709,-0.330513,0.173843,0.097807,-0.438939,...,0.307055,-0.335232,0.200385,-0.227034,-0.169574,0.169691,-0.381725,-0.275363,-0.253356,0.0
4,4,-0.061785,0.041214,-0.007492,-0.095529,0.144389,-0.073329,0.065868,0.044003,-0.194689,...,-0.236539,-0.111709,0.097663,-0.161093,0.065875,0.131907,0.097411,-0.127102,0.054436,0.0
5,5,-0.172550,0.065910,0.097618,0.027284,0.099513,-0.087045,0.022871,0.008577,-0.073297,...,-0.074112,0.069528,-0.004766,-0.315807,-0.023256,0.189096,-0.124768,-0.143838,-0.040053,0.0
6,6,0.065295,0.188122,0.264526,0.052438,-0.016906,-0.017192,0.033818,-0.077562,0.076214,...,-0.019048,-0.215798,0.030529,-0.148937,0.028558,0.103843,0.047945,-0.282794,0.000171,0.0
7,7,-0.019264,0.146217,0.101780,-0.020692,-0.095936,0.039669,0.041793,-0.085271,-0.041021,...,-0.079179,-0.110792,0.133936,-0.149333,0.128809,0.090958,0.012027,-0.149558,-0.044397,0.0
8,8,0.078102,0.024764,0.222488,-0.007545,0.110410,-0.070361,0.165535,0.018249,0.074530,...,0.027959,-0.215296,0.111713,-0.248692,0.051753,0.216604,-0.059347,-0.282305,-0.098882,0.0
9,9,-0.053129,0.143064,0.090162,0.018056,0.074839,-0.001311,0.040774,-0.048419,-0.068145,...,-0.148575,-0.028270,0.095499,-0.127128,-0.006080,0.205171,0.047120,-0.112574,0.060608,0.0
